In [1]:
import os
from typing import Dict, List

# %%
# 项目配置
class Config:
    # 数据集配置
    DATA_PATH = '/mnt/MCP/hagrid_classification'
    CLASSES = ['call', 'fist', 'like', 'ok', 'one', 'palm', 'rock', 'three', 'two_up']
    NUM_CLASSES = len(CLASSES)
    
    # 模型配置
    MODEL_NAME = 'MobileNetV2'
    INPUT_SIZE = (3, 224, 224)
    USE_PRETRAINED = True
    
    # 训练配置
    BATCH_SIZE = 32
    NUM_EPOCHS = 100
    LEARNING_RATE = 0.001  # MobileNet通常使用略高的学习率
    WEIGHT_DECAY = 0.0001  # 减少权重衰减，避免过度正则化
    NUM_WORKERS = 4
    
    # 早停配置
    PATIENCE = 15
    MIN_DELTA = 0.001
    
    # 模型保存路径
    MODEL_SAVE_DIR = './models'
    BEST_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, 'best_mobilenetv2_gesture_model.pth')
    FINAL_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, 'final_mobilenetv2_gesture_model.pth')

# 创建保存模型的目录
if not os.path.exists(Config.MODEL_SAVE_DIR):
    os.makedirs(Config.MODEL_SAVE_DIR)

# %%
# 数据集统计工具
class DatasetAnalyzer:
    def __init__(self, data_path: str, classes: List[str]):
        self.data_path = data_path
        self.classes = classes
        self.image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
        
        # 初始化路径
        self.train_dir = os.path.join(data_path, 'train')
        self.val_dir = os.path.join(data_path, 'val')
        
        # 统计结果
        self.train_counts: Dict[str, int] = {}
        self.val_counts: Dict[str, int] = {}
        self.total_counts: Dict[str, int] = {}
        self.ratio: Dict[str, float] = {}  # 各类别占总样本比例(%)
        
    def _is_image_file(self, filename: str) -> bool:
        """判断文件是否为图像文件"""
        ext = os.path.splitext(filename)[1].lower()
        return ext in self.image_extensions
    
    def _count_class_samples(self, root_dir: str) -> Dict[str, int]:
        """统计指定目录下各类别的样本数量"""
        counts = {cls: 0 for cls in self.classes}
        if not os.path.exists(root_dir):
            return counts
            
        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)
            if not os.path.exists(cls_dir):
                continue
            # 统计目录下所有图像文件
            for filename in os.listdir(cls_dir):
                if self._is_image_file(filename):
                    counts[cls] += 1
        return counts
    
    def analyze(self) -> None:
        """执行数据集分析"""
        print("开始分析数据集...")
        self.train_counts = self._count_class_samples(self.train_dir)
        self.val_counts = self._count_class_samples(self.val_dir)
        
        # 计算总样本数和比例
        total_samples = sum(self.train_counts.values()) + sum(self.val_counts.values())
        self.total_counts = {
            cls: self.train_counts[cls] + self.val_counts[cls] 
            for cls in self.classes
        }
        self.ratio = {
            cls: (self.total_counts[cls] / total_samples) * 100 
            if total_samples > 0 else 0 
            for cls in self.classes
        }
        print("数据集分析完成")
    
    def generate_report(self) -> str:
        """生成论文可用的数据集报告"""
        train_total = sum(self.train_counts.values())
        val_total = sum(self.val_counts.values())
        total = train_total + val_total
        train_ratio = (train_total / total) * 100 if total > 0 else 0
        val_ratio = 100 - train_ratio
        
        # 构建报告
        report = []
        report.append("## 数据集详情报告")
        report.append("### 1. 数据集概述")
        report.append(f"本研究使用的手势数据集存储于路径：`{self.data_path}`，主要用于手势分类任务，包含{Config.NUM_CLASSES}种常见手势类别。")
        report.append(f"数据集采用训练-验证划分方式，共包含{total:,}个样本，其中训练集{train_total:,}个样本({train_ratio:.1f}%)，验证集{val_total:,}个样本({val_ratio:.1f}%)。")
        
        report.append("\n### 2. 类别信息")
        report.append(f"数据集包含的{Config.NUM_CLASSES}个手势类别分别为：{', '.join(self.classes)}。")
        
        report.append("\n### 3. 样本分布")
        report.append("各类别的样本数量及分布比例如下表所示：")
        
        # 生成Markdown表格
        report.append("\n| 类别 | 训练集样本数 | 验证集样本数 | 总样本数 | 占比(%) |")
        report.append("|------|--------------|--------------|----------|---------|")
        for cls in self.classes:
            report.append(f"| {cls} | {self.train_counts[cls]:,} | {self.val_counts[cls]:,} | {self.total_counts[cls]:,} | {self.ratio[cls]:.1f} |")
        
        # 分布均衡性分析
        max_count = max(self.total_counts.values()) if self.total_counts else 0
        min_count = min(self.total_counts.values()) if self.total_counts else 0
        balance_ratio = min_count / max_count if max_count > 0 else 0
        balance_note = "样本分布较为均衡" if balance_ratio > 0.7 else "样本分布存在一定不均衡性"
        report.append(f"\n由表可知，{balance_note}，最大类别样本数为{max_count:,}，最小类别样本数为{min_count:,}，两者比例为{balance_ratio:.2f}:1。")
        
        report.append("\n### 4. 数据格式")
        report.append(f"所有样本均为图像文件，支持常见格式({', '.join(self.image_extensions)})，训练时将统一处理为{Config.INPUT_SIZE[1]}×{Config.INPUT_SIZE[2]}像素的三通道图像。")
        
        return "\n".join(report)

# %%
# 执行分析并生成报告
if __name__ == "__main__":
    # 检查数据集路径
    if not os.path.exists(Config.DATA_PATH):
        print(f"错误: 数据路径 {Config.DATA_PATH} 不存在，请确认数据集路径！")
    else:
        print(f"开始处理数据集: {Config.DATA_PATH}")
        analyzer = DatasetAnalyzer(Config.DATA_PATH, Config.CLASSES)
        analyzer.analyze()
        
        # 生成并打印报告
        dataset_report = analyzer.generate_report()
        print("\n" + "="*50)
        print("数据集情况报告（可直接用于论文）：")
        print("="*50 + "\n")
        print(dataset_report)
        
        # 保存报告到文件
        with open("dataset_report.txt", "w", encoding="utf-8") as f:
            f.write(dataset_report)
        print("\n报告已保存至 dataset_report.txt")

开始处理数据集: /mnt/MCP/hagrid_classification
开始分析数据集...
数据集分析完成

数据集情况报告（可直接用于论文）：

## 数据集详情报告
### 1. 数据集概述
本研究使用的手势数据集存储于路径：`/mnt/MCP/hagrid_classification`，主要用于手势分类任务，包含9种常见手势类别。
数据集采用训练-验证划分方式，共包含231,177个样本，其中训练集204,187个样本(88.3%)，验证集26,990个样本(11.7%)。

### 2. 类别信息
数据集包含的9个手势类别分别为：call, fist, like, ok, one, palm, rock, three, two_up。

### 3. 样本分布
各类别的样本数量及分布比例如下表所示：

| 类别 | 训练集样本数 | 验证集样本数 | 总样本数 | 占比(%) |
|------|--------------|--------------|----------|---------|
| call | 20,034 | 3,000 | 23,034 | 10.0 |
| fist | 23,480 | 2,999 | 26,479 | 11.5 |
| like | 23,197 | 2,999 | 26,196 | 11.3 |
| ok | 23,131 | 2,999 | 26,130 | 11.3 |
| one | 23,034 | 3,000 | 26,034 | 11.3 |
| palm | 23,654 | 2,999 | 26,653 | 11.5 |
| rock | 22,311 | 2,997 | 25,308 | 10.9 |
| three | 22,685 | 2,998 | 25,683 | 11.1 |
| two_up | 22,661 | 2,999 | 25,660 | 11.1 |

由表可知，样本分布较为均衡，最大类别样本数为26,653，最小类别样本数为23,034，两者比例为0.86:1。

### 4. 数据格式
所有样本均为图像文件，支持常见格式(.bmp, .jpg, .jpeg, .png, .gif)，训练时将统一处理为224×224像素的三通道图像。

报告已保存至 da